Telemetry analysis for unified-urlbar experiment.
https://bugzilla.mozilla.org/show_bug.cgi?id=1219505

In [1]:
import pandas as pd
import ujson as json
import numpy as np

%pylab inline

Populating the interactive namespace from numpy and matplotlib


Read previously stored experiment data.

In [2]:
s3_path = "s3n://net-mozaws-prod-us-west-2-pipeline-analysis/mak/unified-urlbar/v1/"
data = sc.textFile(s3_path)
data.count()

255483

In [3]:
experiment_data = data.map(lambda d: eval(d))
experiment_data.first()

{'clientId': u'37e652d8-e938-4e67-8208-7456edd2df9e',
 'environment/addons/activeExperiment/branch': u'unified',
 'environment/addons/activeExperiment/id': u'unified-urlbar@experiments.mozilla.org',
 'environment/settings/defaultSearchEngine': u'google',
 'environment/settings/userPrefs/browser.urlbar.suggest.searches': None,
 'environment/settings/userPrefs/browser.urlbar.userMadeSearchSuggestionsChoice': None,
 'payload/histograms/FX_URLBAR_SELECTED_RESULT_TYPE': 0,
 'payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings': None,
 'payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/urlbar/search-settings': None,
 'payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search-oneoff': None,
 'payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/abouthome': None,
 'payload/simpleMeasurements/UITelemetry/toolbars/countableEvents

Normalize number of samples.

In [4]:
c = experiment_data.map(lambda d: (d["environment/addons/activeExperiment/branch"])).countByValue()
normalized_c = min(c["control"], c["unified"])
control_data = sc.parallelize(experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "control").take(normalized_c))
unified_data = sc.parallelize(experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "unified").take(normalized_c))
customized_data = experiment_data.filter(lambda d: d["environment/addons/activeExperiment/branch"] == "customized")

Analyze data.

In [5]:
class Accumulator():
    def __init__(self, cx):
        # number of analyzed pings
        self.ping_count = cx.accumulator(0)
        # number of searches from the search bar
        self.searchbar_searches = cx.accumulator(0)
        # search coming from urlbar oneoff
        self.from_searchbar_oneoff = cx.accumulator(0)
        # number of searches from the urlbar
        self.urlbar_searches = cx.accumulator(0)
        # number of searches from the about:home
        self.home_searches = cx.accumulator(0)
        # number of searches from the about:newtab
        self.newtab_searches = cx.accumulator(0)
        # search coming from urlbar suggestion
        self.from_urlbar_suggestion = cx.accumulator(0)
        # search coming from urlbar oneoff
        self.from_urlbar_oneoff = cx.accumulator(0)
        # has urlbar suggestions enabled
        self.has_suggestions = cx.accumulator(0)
        # made urlbar suggestions choice
        self.made_choice = cx.accumulator(0)
        # disabled urlbar suggestions
        self.disabled_suggestions = cx.accumulator(0)
        # search bar on nav-bar
        self.has_searchbar = cx.accumulator(0)
        # did any search
        self.ping_has_searches = cx.accumulator(0)
        # total number of searches
        self.total_searches = cx.accumulator(0)
        # searchbar settings
        self.searchbar_settings = cx.accumulator(0)
        # urlbar settings
        self.urlbar_settings = cx.accumulator(0)

def process_data(acc, d):
    acc.ping_count.add(1)

    has_suggestions = d["environment/settings/userPrefs/browser.urlbar.suggest.searches"]
    if has_suggestions is True:
        acc.has_suggestions.add(1)

    made_choice = d["environment/settings/userPrefs/browser.urlbar.userMadeSearchSuggestionsChoice"]
    if made_choice is True:
        acc.made_choice.add(1)
        if has_suggestions is not True:
            acc.disabled_suggestions.add(1)

    ping_has_searches = False
    searchbar_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/searchbar"]
    if searchbar_searches is not None:
        acc.searchbar_searches.add(searchbar_searches)
        acc.total_searches.add(searchbar_searches)
        ping_has_searches = True

    urlbar_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/urlbar"]
    if urlbar_searches is not None:
        acc.urlbar_searches.add(urlbar_searches)
        acc.total_searches.add(urlbar_searches)
        ping_has_searches = True

    home_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/abouthome"]
    if home_searches is not None:
        acc.home_searches.add(home_searches)
        acc.total_searches.add(home_searches)
        ping_has_searches = True

    newtab_searches = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search/newtab"]
    if newtab_searches is not None:
        acc.newtab_searches.add(newtab_searches)
        acc.total_searches.add(newtab_searches)
        ping_has_searches = True

    widgetsInDefaultPosition = d["payload/simpleMeasurements/UITelemetry/toolbars/defaultKept"]
    if widgetsInDefaultPosition is not None:
        if widgetsInDefaultPosition:
            acc.has_searchbar.add(1)

    from_urlbar_suggestion = d["payload/histograms/FX_URLBAR_SELECTED_RESULT_TYPE"]
    if from_urlbar_suggestion is not None:
        acc.from_urlbar_suggestion.add(from_urlbar_suggestion)

    oneoff = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/search-oneoff"]
    if oneoff is not None:
        for o in {k:v for (k,v) in oneoff.iteritems() if "urlbar" in k and "oneoff" in k}.values():
            for v in o.values():
                acc.from_urlbar_oneoff.add(sum(v.values()))

        for o in {k:v for (k,v) in oneoff.iteritems() if "urlbar" not in k and "oneoff" in k}.values():
            for v in o.values():
                acc.from_searchbar_oneoff.add(sum(v.values()))
                acc.searchbar_searches.add(sum(v.values()))
                acc.total_searches.add(sum(v.values()))
                ping_has_searches = True

        for o in {k:v for (k,v) in oneoff.iteritems() if "urlbar" not in k and "oneoff" not in k}.values():
            for v in o.values():
                acc.searchbar_searches.add(sum(v.values()))
                acc.total_searches.add(sum(v.values()))
                ping_has_searches = True

    if ping_has_searches is True:
        acc.ping_has_searches.add(1)

    urlbar_settings = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings"]
    if urlbar_settings is not None:
        acc.urlbar_settings.add(urlbar_settings)

    searchbar_settings = d["payload/simpleMeasurements/UITelemetry/toolbars/countableEvents/__DEFAULT__/click-builtin-item/searchbar/search-settings"]
    if searchbar_settings is not None:
        acc.searchbar_settings.add(searchbar_settings)

control = Accumulator(sc)
control_data.foreach(lambda d: process_data(control, d))
unified = Accumulator(sc)
unified_data.foreach(lambda d: process_data(unified, d))
customized = Accumulator(sc)
customized_data.foreach(lambda d: process_data(customized, d))

Print results.

In [6]:
def pval(name, type, a1, a2, a3):
    print ("{:30s} {:>15" + type + "} {:>15" + type + "} {:>15" + type + "}").format(name, a1, a2, a3)

def pper(name, v, t, a1, a2, a3):
    def perc(acc):
          return (float(getattr(acc, v).value) / getattr(acc, t).value) * 100
    print ("{:30s} {:>14.2f}% {:>14.2f}% {:>14.2f}%").format(name, perc(a1), perc(a2), perc(a3))

pval("", "s", "control", "unified", "customized")
pval("number of users", "d", control.ping_count.value, unified.ping_count.value, customized.ping_count.value)
pval("total search volume", "d", control.total_searches.value, unified.total_searches.value, customized.total_searches.value)
pper("users who searched", "ping_has_searches", "ping_count", control, unified, customized)
print ""
#pval("from urlbar", "d", control.urlbar_searches.value, unified.urlbar_searches.value, customized.urlbar_searches.value)
pper("from urlbar", "urlbar_searches", "total_searches", control, unified, customized)
#pval("  from urlbar suggestion", "d", control.from_urlbar_suggestion.value, unified.from_urlbar_suggestion.value, customized.from_urlbar_suggestion.value)
pper("  from suggestion", "from_urlbar_suggestion", "urlbar_searches", control, unified, customized)
#pval("  from urlbar one-off", "d", control.from_urlbar_oneoff.value, unified.from_urlbar_oneoff.value, customized.from_urlbar_oneoff.value)
pper("  from one-off", "from_urlbar_oneoff", "urlbar_searches", control, unified, customized)
print ""
pper("from searchbar", "searchbar_searches", "total_searches", control, unified, customized)
#pval("  from searchbar one-off", "d", control.from_searchbar_oneoff.value, unified.from_searchbar_oneoff.value, customized.from_searchbar_oneoff.value)
pper("  from one-off", "from_searchbar_oneoff", "searchbar_searches", control, unified, customized)
print ""
pper("from about:home", "home_searches", "total_searches", control, unified, customized)
pper("from about:newtab", "newtab_searches", "total_searches", control, unified, customized)
print ""
pper("made suggestions choice", "made_choice", "ping_count", control, unified, customized)
pper("  enabled urlbar suggestions", "has_suggestions", "made_choice", control, unified, customized)
pper("  disabled urlbar suggestions", "disabled_suggestions", "made_choice", control, unified, customized)
print ""
pper("searchbar in nav-bar", "has_searchbar", "ping_count", control, unified, customized)
pper("searchbar settings button", "searchbar_settings", "ping_count", control, unified, customized)
pper("urlbar settings button", "urlbar_settings", "ping_count", control, unified, customized)

                                       control         unified      customized
number of users                         124804          124804            5267
total search volume                     123895           87146            4285
users who searched                      29.07%          27.24%          23.18%

from urlbar                             23.80%          46.93%          62.08%
  from suggestion                        1.34%           1.29%           1.05%
  from one-off                           0.00%           2.21%           0.64%

from searchbar                          47.79%           7.12%          11.11%
  from one-off                           4.16%           2.35%           5.25%

from about:home                         18.08%          27.26%          12.60%
from about:newtab                       10.32%          18.70%          14.21%

made suggestions choice                 23.85%          23.34%          25.40%
  enabled urlbar suggestions            51.68%  